### This notebook serves as the intial exploration of data to build the actual etl.py file

In [220]:
import kagglehub
import os
import shutil
import zipfile
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

In [221]:
# Define paths
RAW_DIR = "../data/raw/"
PROCESSED_DIR = "../data/processed/"
FINAL_CSV_NAME = "medical_insurance.csv"

In [222]:
# Ensure directories exist
os.makedirs(RAW_DIR, exist_ok=True)
os.makedirs(PROCESSED_DIR, exist_ok=True)


In [223]:
# Step 1: Download dataset to kagglehub cache
dataset_path = kagglehub.dataset_download("rahulvyasm/medical-insurance-cost-prediction")
print("Downloaded to cache:", dataset_path)

Downloaded to cache: C:\Users\vaina\.cache\kagglehub\datasets\rahulvyasm\medical-insurance-cost-prediction\versions\1


In [224]:
# Step 2: Move any CSV file to PROCESSED_DIR and rename
csv_found = False
for file in os.listdir(dataset_path):
    if file.endswith(".csv"):
        src_csv_path = os.path.join(dataset_path, file)
        dest_csv_path = os.path.join(RAW_DIR, FINAL_CSV_NAME)
        shutil.copy2(src_csv_path, dest_csv_path)
        print(f"Copied and renamed {file} to {dest_csv_path}")
        csv_found = True
        break

if not csv_found:
    raise FileNotFoundError("No CSV file found in the KaggleHub dataset directory.")

Copied and renamed medical_insurance.csv to ../data/raw/medical_insurance.csv


In [225]:
# Step 4: Rename CSV file if needed
for file in os.listdir(RAW_DIR):
    if file.endswith(".csv") and file != FINAL_CSV_NAME:
        old_path = os.path.join(RAW_DIR, file)
        new_path = os.path.join(RAW_DIR, FINAL_CSV_NAME)
        os.rename(old_path, new_path)
        print(f"Renamed {file} to {FINAL_CSV_NAME}")
        break

## Data Cleaning


In [226]:
csv_path = os.path.join(RAW_DIR, FINAL_CSV_NAME)
df = pd.read_csv(csv_path)

In [227]:
df.head()

age     sex     bmi  children smoker     region      charges
0   19  female  27.900         0    yes  southwest  16884.92400
1   18    male  33.770         1     no  southeast   1725.55230
2   28    male  33.000         3     no  southeast   4449.46200
3   33    male  22.705         0     no  northwest  21984.47061
4   32    male  28.880         0     no  northwest   3866.85520

In [228]:
# check for missing values
missing_values = df.isnull().sum()
print("Missing values in each column:")
print(missing_values)

Missing values in each column:
age         0
sex         0
bmi         0
children    0
smoker      0
region      0
charges     0
dtype: int64


In [229]:
#check for duplicates
duplicates = df.duplicated().sum()
print(f"Number of duplicate rows: {duplicates}")

Number of duplicate rows: 1435


In [230]:
# percentage of duplicate rows
duplicate_percentage = (duplicates / len(df)) * 100
print(f"Percentage of duplicate rows: {duplicate_percentage:.2f}%")

Percentage of duplicate rows: 51.77%


since the percentage is too high and given the context of data we will not drop these rows

### Data Transformations

In [231]:
## Boolean columns for isparent if childern>0
df['isparent'] = df['children'] > 0

In [232]:
max(df['age'])  # Maximum age in the dataset



64

In [233]:
min(df['age'])  # Minimum age in the dataset

18

In [234]:
# categorizing into young adults, adults, and seniors
df['age_group'] = pd.cut(
    df['age'],
    bins=[17, 28, 48, np.inf],  # Adjusted to include age 18
    labels=['young_adult', 'adult', 'senior'],
    right=True,  # Includes upper bound in the bin
    include_lowest=True
)

In [235]:
df.head()  # Display the first few rows of the DataFrame

age     sex     bmi  children smoker     region      charges  isparent  \
0   19  female  27.900         0    yes  southwest  16884.92400     False   
1   18    male  33.770         1     no  southeast   1725.55230      True   
2   28    male  33.000         3     no  southeast   4449.46200      True   
3   33    male  22.705         0     no  northwest  21984.47061     False   
4   32    male  28.880         0     no  northwest   3866.85520     False   

     age_group  
0  young_adult  
1  young_adult  
2  young_adult  
3        adult  
4        adult

In [236]:
## Save the processed DataFrame to a new CSV file for EDA
EDA_FILE_NAME = "medical_insurance_eda.csv"
eda_csv_path = os.path.join(PROCESSED_DIR, EDA_FILE_NAME)
df.to_csv(eda_csv_path, index=False)
print(f"Processed data saved to {eda_csv_path}")

Processed data saved to ../data/processed/medical_insurance_eda.csv


### Now furhter processign for EDA

In [237]:
# Identify categorical columns (object or category dtype)
categorical_columns = df.select_dtypes(include=['object', 'category']).columns

# Count unique values in each categorical column
unique_counts = {col: df[col].nunique() for col in categorical_columns}
print("Unique value counts in categorical columns:")
for col, count in unique_counts.items():
    print(f"{col}: {count} unique values")

Unique value counts in categorical columns:
sex: 2 unique values
smoker: 2 unique values
region: 4 unique values
age_group: 3 unique values


In [238]:
# label encoding for categorical columns
from sklearn.preprocessing import LabelEncoder
label_encoders = {}
for col in categorical_columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le
    print(f"Encoded {col} with {len(le.classes_)} unique values.")

Encoded sex with 2 unique values.
Encoded smoker with 2 unique values.
Encoded region with 4 unique values.
Encoded age_group with 3 unique values.


In [239]:
# label encode isparent column
df['isparent'] = le.fit_transform(df['isparent'])

In [240]:
df.head()

age  sex     bmi  children  smoker  region      charges  isparent  \
0   19    0  27.900         0       1       3  16884.92400         0   
1   18    1  33.770         1       0       2   1725.55230         1   
2   28    1  33.000         3       0       2   4449.46200         1   
3   33    1  22.705         0       0       1  21984.47061         0   
4   32    1  28.880         0       0       1   3866.85520         0   

   age_group  
0          2  
1          2  
2          2  
3          0  
4          0

In [241]:
# scale numerical columns
numerical_columns = df.select_dtypes(include=['int64', 'float64']).columns
scaler = StandardScaler()
df[numerical_columns] = scaler.fit_transform(df[numerical_columns])
print("Scaled numerical columns:", numerical_columns)


Scaled numerical columns: Index(['age', 'sex', 'bmi', 'children', 'smoker', 'region', 'charges',
       'isparent', 'age_group'],
      dtype='object')


In [242]:
df.head()

age       sex       bmi  children    smoker    region   charges  \
0 -1.428353 -1.014536 -0.457114 -0.907084  1.978609  1.330169  0.298245   
1 -1.499381  0.985673  0.500731 -0.083758 -0.505406  0.423771 -0.949483   
2 -0.789099  0.985673  0.375085  1.562893 -0.505406  0.423771 -0.725285   
3 -0.433959  0.985673 -1.304814 -0.907084 -0.505406 -0.482628  0.717976   
4 -0.504987  0.985673 -0.297201 -0.907084 -0.505406 -0.482628 -0.773238   

   isparent  age_group  
0 -1.156403   1.330755  
1  0.864750   1.330755  
2  0.864750   1.330755  
3 -1.156403  -1.089752  
4 -1.156403  -1.089752

In [243]:
# save the final processed DataFrame to a new CSV file
FINAL_PROCESSED_CSV_NAME = "medical_insurance_ML.csv"
final_csv_path = os.path.join(PROCESSED_DIR, FINAL_PROCESSED_CSV_NAME)
df.to_csv(final_csv_path, index=False)

In [244]:
## Save to MySQL database
from dotenv import load_dotenv
import os
load_dotenv()  # load variables from .env file into environment


username = os.getenv('DB_USERNAME')
password = os.getenv('DB_PASSWORD')
host = os.getenv('DB_HOST')
port = int(os.getenv('DB_PORT'))  # This will be an int as intended
database = os.getenv('DB_DATABASE')


In [245]:
from sqlalchemy import create_engine
engine = create_engine(f'mysql+pymysql://{username}:{password}@{host}:{port}/{database}')


In [246]:
df = pd.read_csv(final_csv_path)
df.to_sql('insurance_ml', con=engine, if_exists='replace', index=False)
print("Data successfully loaded into MySQL!")

Data successfully loaded into MySQL!


In [247]:
df = pd.read_csv(eda_csv_path)
df.to_sql('insurance_bi', con=engine, if_exists='replace', index=False)
print("Data successfully loaded into MySQL!")

Data successfully loaded into MySQL!
